<a href="https://colab.research.google.com/github/mahtabkarami/Projects/blob/main/music_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install music21 tensorflow


In [ ]:
from google.colab import files

# Prompt for file upload
uploaded = files.upload()





Saving Erica_Glenn_-_Blank_Canvas.mid to Erica_Glenn_-_Blank_Canvas (2).mid


In [ ]:
import numpy as np
import music21 as m21
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import os

# Load MIDI files and extract notes
def load_notes_from_midi(midi_files):
    notes = []
    for file in midi_files:
        midi = m21.converter.parse(file)
        notes_to_parse = None

        # Parse the MIDI file
        if hasattr(midi, 'flat'):
            notes_to_parse = midi.flat.notes

        for element in notes_to_parse:
            if isinstance(element, m21.note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, m21.chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))
    return notes

# Upload your MIDI files
uploaded = files.upload()
midi_files = list(uploaded.keys())

# Extract notes from the uploaded MIDI files
notes = load_notes_from_midi(midi_files)

# Create a mapping from unique notes to integers
unique_notes = sorted(set(notes))
note_to_int = {note: number for number, note in enumerate(unique_notes)}

# Prepare the dataset for training
sequence_length = 100
network_input = []
network_output = []

for i in range(len(notes) - sequence_length):
    seq_in = notes[i:i + sequence_length]
    seq_out = notes[i + sequence_length]
    network_input.append([note_to_int[note] for note in seq_in])
    network_output.append(note_to_int[seq_out])

n_patterns = len(network_input)
n_vocab = len(unique_notes)

# Reshape input for LSTM [samples, time steps, features]
X = np.reshape(network_input, (n_patterns, sequence_length, 1))
X = X / float(n_vocab)  # Normalize input
y = keras.utils.to_categorical(network_output)

# Build LSTM model
model = keras.Sequential()
model.add(layers.LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(layers.Dropout(0.3))
model.add(layers.LSTM(256))
model.add(layers.Dense(256))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(n_vocab, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

# Train the model (adjust epochs and batch size as needed)
model.fit(X, y, epochs=100, batch_size=64)

# Function to generate music
def generate_music(model, start_note, num_notes=100):
    int_to_note = {number: note for number, note in enumerate(unique_notes)}

    pattern = [note_to_int[note] for note in start_note]
    generated_notes = []

    for _ in range(num_notes):
        input_pattern = np.reshape(pattern, (1, len(pattern), 1))
        input_pattern = input_pattern / float(n_vocab)

        prediction = model.predict(input_pattern, verbose=0)
        index = np.argmax(prediction)
        result = int_to_note[index]
        generated_notes.append(result)

        pattern.append(index)
        pattern = pattern[1:]

    return generated_notes
# Generate new music starting from a sequence of notes
start_sequence = notes[:sequence_length]  # Use the first sequence from the training data
generated_music = generate_music(model, start_sequence)

# Convert generated notes back to a MIDI file
stream = m21.stream.Stream()
for note in generated_music:
    if '.' in note or note.isdigit():  # It's a chord
        notes_in_chord = note.split('.')
        chord_notes = [m21.note.Note(int(n)) for n in notes_in_chord]
        new_chord = m21.chord.Chord(chord_notes)
        stream.append(new_chord)
    else:  # It's a note
        new_note = m21.note.Note(note)
        stream.append(new_note)

# Save the generated music as a MIDI file
stream.write('midi', fp='generated_music.mid')

print("Generated music saved as 'generated_music.mid'")


Saving Erica_Glenn_-_Blank_Canvas.mid to Erica_Glenn_-_Blank_Canvas (3).mid


/usr/local/lib/python3.11/dist-packages/music21/stream/base.py:3689: Music21DeprecationWarning: .flat is deprecated.  Call .flatten() instead
  return self.iter().getElementsByClass(classFilterList)
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 20s 1s/step - loss: 3.4515
Epoch 2/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 20s 1s/step - loss: 3.2475
Epoch 3/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 20s 1s/step - loss: 3.2603
Epoch 4/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 21s 1s/step - loss: 3.2312
Epoch 5/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 20s 1s/step - loss: 3.2009
Epoch 6/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 20s 1s/step - loss: 3.2088
Epoch 7/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 21s 1s/step - loss: 3.1685
Epoch 8/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 21s 1s/step - loss: 3.1685
Epoch 9/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 23s 1s/step - loss: 3.2224
Epoch 10/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 18s 1s/step - loss: 3.1510
Epoch 11/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 21s 1s/step - loss: 3.1976
Epoch 12/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 20s 1s/step - loss: 3.1809
Epoch 13/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 21s 1s/step - loss: 3.1821
Epoch 14/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 20s 1s/step - loss: 3.1860
Epoch 15/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 21s 1s/step - loss: 3.1857
Epoc